In [2]:
import torch
import torch.nn as nn

In [3]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.first_layer = nn.Linear(10, 1)

    def forward(self, x):
        x = self.first_layer(x)
        return x

In [4]:
m = MyModel()

# Train the model:
optimizer = torch.optim.Adam(m.parameters(), lr=0.001)
loss_function = nn.MSELoss() # or any other appropriate loss function

In [14]:
# 1000 times
loss = 0
for i in range(1000):
  x = torch.rand(32, 10) # 32: batch size
  target = torch.ones(32,1)
  y = m(x)

  loss = loss_function(y, target)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

print('loss : ', loss)

loss :  tensor(4.6268e-06, grad_fn=<MseLossBackward0>)


In [19]:
# save the weights:
torch.save(m.state_dict(), 'my_model_state_dict.pth')

In [22]:
# load the model from weights:
m2 = MyModel()
m2.load_state_dict(torch.load('my_model_state_dict.pth'))

x = torch.rand(1, 10)
y = m2(x)
print(y)

tensor([[1.0011]], grad_fn=<AddmmBackward0>)
